In [1]:
from pyspark.sql.functions import current_timestamp

In [2]:
from pyspark.sql import SparkSession

# Инициализируем Spark Session с добавлением PostgreSQL JDBC драйвера
spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("JupyterSparkCluster") \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.5.4") \
    .getOrCreate()
spark

In [3]:
# Параметры подключения к PostgreSQL
jdbc_url = "jdbc:postgresql://postgres:5432/postgres"
connection_properties = {
    "user": "postgres",
    "password": "postgres",
    "driver": "org.postgresql.Driver"
}


In [4]:
def load_table(spark, jdbc_url, connection_properties, source_table, target_table, selected_columns, mode='append'):
    """
    Загружает данные из исходной таблицы в целевую таблицу DWH с выборкой необходимых колонок.

    :param spark: Объект SparkSession.
    :param jdbc_url: JDBC URL для подключения к PostgreSQL.
    :param connection_properties: Словарь с параметрами подключения (user, password, driver).
    :param source_table: Имя исходной таблицы (схема.таблица).
    :param target_table: Имя целевой таблицы в DWH (схема.таблица).
    :param selected_columns: Список колонок для выборки из исходной таблицы.
    :param mode: Режим записи ('overwrite' или 'append'). По умолчанию 'append'.
    """

    # Чтение данных из исходной таблицы
    df = spark.read.jdbc(
        url=jdbc_url,
        table=source_table,
        properties=connection_properties
    )
    
    # Выбор необходимых колонок
    df_selected = df.select(*selected_columns)
    
    # Добавление колонки с временем загрузки
    df_selected = df_selected.withColumn("load_dttm", current_timestamp())
    
    # Запись данных в целевую таблицу
    df_selected.write.jdbc(
        url=jdbc_url,
        table=target_table,
        mode=mode,
        properties=connection_properties
    )
    
    print(f"Таблица {target_table} успешно загружена.")
    return df_selected
        

In [5]:
# Параметры для загрузки dwh.d_craftsmans
source_table_craftsmans = "source3.craft_market_craftsmans"
target_table_craftsmans = "dwh.d_craftsmans"
selected_columns_craftsmans = [
    # "craftsman_id",
    "craftsman_name",
    "craftsman_address",
    "craftsman_birthday",
    "craftsman_email"
]

# Вызов функции для загрузки dwh.d_craftsmans
dwh_d_craftsmans = load_table(
    spark=spark,
    jdbc_url=jdbc_url,
    connection_properties=connection_properties,
    source_table=source_table_craftsmans,
    target_table=target_table_craftsmans,
    selected_columns=selected_columns_craftsmans,
    mode='append'
)

Таблица dwh.d_craftsmans успешно загружена.


In [6]:
# Параметры для загрузки dwh.d_customers
source_table_customers = "source3.craft_market_customers"
target_table_customers = "dwh.d_customers"
selected_columns_customers = [
    # "customer_id",
    "customer_name",
    "customer_address",
    "customer_birthday",
    "customer_email"
]

# Вызов функции для загрузки dwh.d_customers
dwh_d_customers = load_table(
    spark=spark,
    jdbc_url=jdbc_url,
    connection_properties=connection_properties,
    source_table=source_table_customers,
    target_table=target_table_customers,
    selected_columns=selected_columns_customers,
    mode='append'
)

Таблица dwh.d_customers успешно загружена.


In [7]:
dwh_d_customers.head(5)

[Row(customer_name='Anna-maria Lamba', customer_address='8 Prairieview Alley', customer_birthday=datetime.date(1990, 7, 27), customer_email='alamba1@ted.com', load_dttm=datetime.datetime(2024, 12, 26, 4, 5, 13, 700602)),
 Row(customer_name='Kim Coonihan', customer_address='3 Hanson Center', customer_birthday=datetime.date(1992, 5, 26), customer_email='kcoonihan2@angelfire.com', load_dttm=datetime.datetime(2024, 12, 26, 4, 5, 13, 700602)),
 Row(customer_name='Lanny Vasse', customer_address='9 Westridge Alley', customer_birthday=datetime.date(2003, 6, 12), customer_email='lvasse3@pbs.org', load_dttm=datetime.datetime(2024, 12, 26, 4, 5, 13, 700602)),
 Row(customer_name='Alice Treneman', customer_address='68462 Meadow Valley Drive', customer_birthday=datetime.date(1995, 7, 26), customer_email='atreneman4@cmu.edu', load_dttm=datetime.datetime(2024, 12, 26, 4, 5, 13, 700602)),
 Row(customer_name='Flo Morant', customer_address='2453 Crescent Oaks Avenue', customer_birthday=datetime.date(1997

In [8]:
# Параметры для загрузки dwh.d_products
source_table_products = "source2.craft_market_masters_products"  # Предполагается, что products находятся здесь
target_table_products = "dwh.d_products"
selected_columns_products = [
    # "product_id",
    "product_name",
    "product_description",
    "product_type",
    "product_price"
]

# Вызов функции для загрузки dwh.d_products
dwh_d_products = load_table(
    spark=spark,
    jdbc_url=jdbc_url,
    connection_properties=connection_properties,
    source_table=source_table_products,
    target_table=target_table_products,
    selected_columns=selected_columns_products,
    mode='append'
)

Таблица dwh.d_products успешно загружена.


In [9]:
source_table_orders = "source1.craft_market_wide"
target_table_orders = "dwh.f_orders"
selected_columns_orders = [
    "order_created_date",
    "order_completion_date",
    "order_status",
    "customer_id",
    "craftsman_id",
    "product_id",
    "order_id",
]

# Вызов функции для загрузки dwh.f_orders
dwh_f_orders = load_table(
    spark=spark,
    jdbc_url=jdbc_url,
    connection_properties=connection_properties,
    source_table=source_table_orders,
    target_table=target_table_orders,
    selected_columns=selected_columns_orders,
    mode='append'  # Используем 'append' для добавления данных
)

Таблица dwh.f_orders успешно загружена.
